In [144]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [145]:
from notebook_utils import load_results, plot_results, load_all_results, load_results_for_one_checkpoint

In [146]:
%%time
results = load_all_results()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9648/9648 [06:51<00:00, 23.44it/s]

CPU times: user 3min 21s, sys: 5.16 s, total: 3min 26s
Wall time: 6min 51s


In [147]:
from glob import glob
results_mbert = load_results_for_one_checkpoint("finetune/bert-base-multilingual-cased", "bert-base-multilingual-cased", 0)

In [148]:
all_results = results+results_mbert

In [149]:
import pandas as pd

In [150]:
df = pd.DataFrame(all_results)

In [175]:
average_df = df[df.lang == "cs-uk"]
average_df = average_df.drop(columns="lang").groupby(["experiment_name", "step"]).mean().reset_index()
best_steps = average_df.loc[average_df.groupby('experiment_name').aer.idxmin()]
best_steps

,experiment_name,step,aer,precision,recall,f1
0,bert-base-multilingual-cased,0,0.109426,0.882175,0.900352,0.891171
23,mbert_full8M_1epoch,16000,0.087171,0.907312,0.919109,0.913172
103,mbert_full8M_1epoch_finegrained_bs16,5000,0.085637,0.903226,0.927315,0.915112
180,mbert_multilingual_1M-per-lang,8400,0.080742,0.911224,0.928488,0.919775
905,mbert_multilingual_1M-per-lang_only_tlm,333000,0.095987,0.896065,0.913247,0.904574
912,mbert_multilingual_1M-per-lang_only_tlm_add_so,1200,0.081389,0.913131,0.924971,0.919013
1031,mbert_multilingual_1M-per-lang_only_tlm_add_so...,5000,0.079978,0.914721,0.926143,0.920396
1147,mbert_multilingual_1M-per-lang_only_tlm_add_so...,8200,0.080130,0.911469,0.929660,0.920475


In [176]:
allowed = list(best_steps[["experiment_name", "step"]].apply(tuple, axis=1))
condition = df[['experiment_name', 'step']].apply(tuple, axis=1).isin(allowed)
p = df[condition].pivot(index=("experiment_name", "step"), columns="lang", values="aer")*100
p.style.highlight_min(color='lightgreen', axis = 0).format('{:.1f}')

,lang,cs-uk,de-en,en-cs,en-fr,es-fr,ja-en,ro-en,zh-en
experiment_name,step,,,,,,,,
bert-base-multilingual-cased,0,10.9,17.4,9.5,5.6,14.4,45.6,27.9,17.9
mbert_full8M_1epoch,16000,8.7,16.1,7.3,5.1,15.3,47.8,28.0,16.3
mbert_full8M_1epoch_finegrained_bs16,5000,8.6,15.6,6.9,4.5,14.5,44.9,25.3,15.3
mbert_multilingual_1M-per-lang,8400,8.1,15.3,6.7,4.9,14.1,44.5,25.7,14.9
mbert_multilingual_1M-per-lang_only_tlm,333000,9.6,16.0,7.5,4.3,13.9,47.6,39.4,17.0
mbert_multilingual_1M-per-lang_only_tlm_add_so,1200,8.1,14.6,6.1,3.9,13.8,46.0,33.5,14.8
mbert_multilingual_1M-per-lang_only_tlm_add_so_lr1e-5,5000,8.0,14.4,5.8,3.9,13.9,46.2,32.5,14.6
mbert_multilingual_1M-per-lang_only_tlm_add_so_lr5e-6,8200,8.0,14.4,5.8,3.9,13.8,46.1,33.0,14.6
